In [1]:
import sys
import numpy as np
from os.path import abspath, join
sys.path.append(abspath(join('../..')))

from math import pi

from LevelSetPy.Utilities import *
from LevelSetPy.BoundaryCondition import *
from LevelSetPy.Visualization import *
# from LevelSetPy.DynamicalSystems import *
from LevelSetPy.Grids import *
from LevelSetPy.InitialConditions import *
from LevelSetPy.SpatialDerivative import *
from LevelSetPy.ExplicitIntegration.Dissipation import *

# from LargeBRAT.BRATSolver.brt_solver import solve_brt

In [2]:

obj = Bundle({})

pdDims = 2; N = 100
v, w = +1, +1
obj.ve, obj.vp = v, v
obj.we, obj.wp = -w, w

In [3]:

# get player (pursuer) 1's state space
gmin = np.array(([[-5, -5, -pi]])).T
gmax = np.array(([[5, 5, pi]])).T
obj.p1 = Bundle({'pursuer':Bundle({}), 'evader':Bundle({})})
obj.p1.pursuer.grid = createGrid(gmin, gmax, N, pdDims)
obj.p1.pursuer.center = 0

# get player (evader) 2's state space
gmin = np.array(([[-3, -3, -2*pi]])).T
gmax = np.array(([[7, 7, 2*pi]])).T
obj.p1.evader.grid = createGrid(gmin, gmax, N, pdDims)
obj.p1.evader.center = np.array(([[1.5, 1.5, 4*pi/2]]),dtype=np.float64).T

# get player (pursuer) 3's state space
gmin = np.array(([[1, 1, 2]])).T
gmax = np.array(([[6, 6, 8]])).T
obj.p2 = Bundle({'pursuer':Bundle({}), 'evader':Bundle({})})
obj.p2.pursuer.grid = createGrid(gmin, gmax, N, pdDims)
obj.p2.pursuer.center = np.array(([[3.5, 3.5, 4]]),dtype=np.float64).T

# get player (evader) 4's state space
gmin = np.array(([[3, 3, 2*pi/3]])).T
gmax = np.array(([[3*pi, 3*pi, 4*pi]])).T
obj.p2.evader.grid = createGrid(gmin, gmax, N, pdDims)
obj.p2.evader.center = np.array(([[(3*pi-3)/2., (3*pi-3)/2., (4.0-2/3)*(pi/2.0)]]),dtype=np.float64).T



In [4]:
p1, p2, mode= obj.p1.pursuer, obj.p1.evader, 'capture'

assert isfield(p1, 'center'), 'player I must have a center '\
                                'defined for its capture equation.'
assert isfield(p2, 'center'), 'player II must have a center '\
                                'defined for its capture equation.'
assert isfield(p1, 'grid'), 'player I must have its grid info'
assert isfield(p2, 'grid'), 'player II must have its grid info'

x1 = p1.grid.xs
x2 = p2.grid.xs

p1_dyn  = cell(3)
p2_dyn = cell(3)

p1_dyn[0] = obj.vp*np.cos(x1[2])
p1_dyn[1] = obj.vp*np.sin(x1[2])
p1_dyn[2] = obj.we

p2_dyn[0] = obj.ve*np.cos(x2[2])
p2_dyn[1] = obj.ve*np.sin(x2[2])
p2_dyn[2] = obj.wp

data_agent1 = np.zeros(p1.grid.shape)
data_agent2 = np.zeros(p2.grid.shape)

if isfield(p1, 'center') and numel(p1.center==1):
    p1.center = p1.center*np.ones((p1.grid.dim,1), dtype=np.float64)
if isfield(p2, 'center') and numel(p2.center==1):
    p2.center = p2.center*np.ones((p2.grid.dim,1), dtype=np.float64)
        

### This for the Double Integrator

In [1]:
import cupy as cp
import numpy as np

import sys
from os.path import abspath, join
sys.path.append(abspath(join('../..')))
from LevelSetPy.Utilities import *
from LevelSetPy.Grids import createGrid
from LevelSetPy.Helper import postTimeStepTTR
from LevelSetPy.Visualization import implicit_mesh
from LevelSetPy.DynamicalSystems import DoubleIntegrator
from LevelSetPy.SpatialDerivative import upwindFirstWENO5
from LevelSetPy.ExplicitIntegration import artificialDissipationGLF
from LevelSetPy.ExplicitIntegration.Integration import odeCFL3, odeCFLset
from LevelSetPy.ExplicitIntegration.Term import termRestrictUpdate, termLaxFriedrichs


In [2]:
gmin = np.array(([[-1, -1]]),dtype=np.float64).T
gmax = np.array(([[1, 1]]),dtype=np.float64).T
g = createGrid(gmin, gmax, 101, None)

eps_targ = 1.0
u_bound = 1
target_rad = .2 #eps_targ * np.max(g.dx)
dint = DoubleIntegrator(g, u_bound)
value_time2reach = dint.min_time2reach() - target_rad
value_time2reach = np.maximum(0, value_time2reach)

In [5]:
above_curve=dint.grid.xs[0]>dint.Gamma
below_curve=dint.grid.xs[0]<dint.Gamma
on_curve=dint.grid.xs[0]==dint.Gamma

reach_term1  = (dint.grid.xs[1] + np.emath.sqrt(4*dint.grid.xs[0] + \
                         2 * dint.grid.xs[1]**2))*above_curve
reach_term2 =  (-dint.grid.xs[1]+np.emath.sqrt(-4*dint.grid.xs[0] + \
                2 * dint.grid.xs[1]**2) )*below_curve
reach_term3 = np.abs(dint.grid.xs[1]) * on_curve
reach_time = reach_term1.real + reach_term2.real + reach_term3

In [6]:
eps_targ * np.max(g.dx)

0.02

In [7]:
np.linalg.norm(reach_time-target_rad)

162.46702711404032